# Machine learning model LaQuiniela

In this notebook we are going to develop a machine learning model to predict the matches results of LaLiga: (1) home, (2) visitor, (X) tie.

First of all, we import all the libraries we are going to use. 
The library `sqlite3` is for reading the files `.sqlite`, in particular, `laliga.sqlite`, with the information of the matches, and `clasification.sqlite`, where we have store the data from exercise 10.

The library `sklearn` is for the machine learning model. The variable which will be predicted is discrete, so we have to develop a clasification model. To do that we have selected the `RandomForestClassifier`. The other functions of this library will help us to mesure the fitness of our model.

We have also add a new variable, `DATABASE_PATH_1`, to the file `settings.py`, to get the dataframe from exercise 10 with the clssification of each team.

In [85]:
import pandas as pd
import numpy as np
import sqlite3
import sys
sys.path.append('/home/la-quiniela-main/')
import settings
import argparse
from settings import DATABASE_PATH, DATABASE_PATH_1
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

Secondly, we have define every function to clean and merge our two dataframes, matches (laliga) and classification, to get the training data and the test data for our model. 

The first four functions are from the file `io.py` to load the dataframes.

In [86]:
def load_matchday_laliga(season, division, matchday):
    with sqlite3.connect(DATABASE_PATH) as conn:
        data = pd.read_sql(f"""
            SELECT * FROM Matches
                WHERE season = '{season}'
                  AND division = {division}
                  AND matchday = {matchday}
        """, conn)
    if data.empty:
        raise ValueError("There is no matchday data for the values given")
    return data

def load_matchday_clasification(season, division, matchday):
    with sqlite3.connect(DATABASE_PATH_1) as conn:
        data = pd.read_sql(f"""
            SELECT * FROM clasification
                WHERE season = '{season}'
                  AND division = {division}
                  AND matchday = {matchday}
        """, conn)
    if data.empty:
        raise ValueError("There is no matchday data for the values given")
    return data

def load_historical_data_laliga(seasons):
    with sqlite3.connect(DATABASE_PATH) as conn:
        if seasons == "all":
            data = pd.read_sql("SELECT * FROM Matches", conn)
        else:
            data = pd.read_sql(f"""
                SELECT * FROM Matches
                    WHERE season IN {tuple(seasons)}
            """, conn)
    if data.empty:
        raise ValueError(f"No data for seasons {seasons}")
    return data

def load_historical_data_clasification(seasons):
    with sqlite3.connect(DATABASE_PATH_1) as conn:
        if seasons == "all":
            data = pd.read_sql("SELECT * FROM clasification", conn)
        else:
            data = pd.read_sql(f"""
                SELECT * FROM clasification
                    WHERE season IN {tuple(seasons)}
            """, conn)
    if data.empty:
        raise ValueError(f"No data for seasons {seasons}")
    return data


The next function converts the value of the seasons to train, given in the terminal, in a list.

In [87]:
def parse_seasons(value):
    if value == "all":
        return "all"
    seasons = []
    for chunk in value.split(","):
        if ":" in chunk:
            try:
                start, end = map(int, chunk.split(":"))
                assert start < end
            except Exception:
                raise argparse.ArgumentTypeError(f"Unexpected format for seasons {value}")
            for i in range(start, end):
                seasons.append(f"{i}-{i+1}")
        else:
            try:
                start, end = map(int, chunk.split("-"))
                assert start == end - 1
            except Exception:
                raise argparse.ArgumentTypeError(f"Unexpected format for seasons {value}")
            seasons.append(chunk)
    return seasons

Te next one is to save the predictions of our model in the table `Predictions` inside the file `laliga.sqlite`, which is originally empty.

In [88]:
def save_predictions(predictions):
    predictions = predictions[["season", "division", "matchday", 
                               "date", "time", "home_team", 
                               "away_team", "score", "pred"]]
    with sqlite3.connect(DATABASE_PATH) as conn:
        predictions.to_sql(name="Predictions", con=conn, if_exists="append", index=False)

The function `add_result` adds a new column to the matches dataframe. This is going to be our target column, the one we want to predict.

In [89]:
def add_result(df):
    df2 = df.copy()
    df2["result"] = None
    df2.loc[(df2["score"].str.split(":").str[0]) > (df2["score"].str.split(":").str[1]), "result"] = '1'
    df2.loc[(df2["score"].str.split(":").str[0]) == (df2["score"].str.split(":").str[1]), "result"] = 'X'
    df2.loc[(df2["score"].str.split(":").str[0]) < (df2["score"].str.split(":").str[1]), "result"] = '2'
    
    df2.dropna(subset="score", inplace=True)
    return df2

The next two functions merge two dataframes. We are going to used them as follows: first we merge the dataframes matches and classification in a way that we obtain another dataframe, df, with the data of every match and the classification data of the home_team. Then we used the second function, to merge df with the classsification dataframe to obtain a final dataframe with the data of every match and the classification data of both teams, home_team and away_team.

In [90]:
def merge_and_clean_home(df_games, df_teams):
    df = pd.merge(df_games.copy(), df_teams.copy(), 
                             left_on=['home_team', "season", "division", "matchday"],
                             right_on=['team', "season", "division", "matchday"])
    df.drop("team", axis=1, inplace=True)

    return df

In [91]:
def merge_and_clean_visitor(df, df_teams):
    df2 = pd.merge(df.copy(), df_teams.copy(), 
                             left_on=['away_team', "season", "division", "matchday"],
                             right_on=['team', "season", "division", "matchday"],
                             suffixes=("_home", "_away"))
    df2.drop("team", axis=1, inplace=True)

    return df2

Finally, the last two functions create the dataframes we need to train and test our model.

In [92]:
def df_train(seasons):   
    df_teams = load_historical_data_clasification(seasons)
    df_games = load_historical_data_laliga(seasons)
    df_teams.dropna(subset="rank", inplace=True)
    df_games = add_result(df_games)
    
    df_train = merge_and_clean_home(df_games, df_teams)
    df_train = merge_and_clean_visitor(df_train, df_teams)
    return df_train

In [93]:
def df_test(season, division, matchday):
    df_teams = load_matchday_clasification(season, division, matchday)
    df_games =load_matchday_laliga(season, division, matchday)
    df_teams.dropna(subset="rank", inplace=True)
    df_games = add_result(df_games)
    
    df_test = merge_and_clean_home(df_games,df_teams)
    df_test = merge_and_clean_visitor(df_test, df_teams)
    return df_test

# Random Forest Classifier

We have used a Random Forest Classifier model to this problem because it is a classification problem.

The variables of the cell below contains the arguments which are going to be given in the terminal by the user.

In [94]:
seasons = "2000:2010"
season = "2021-2022"
division = 1
matchday = 3 

We have decided to used the following features to our model.

In [95]:
features = ['rank_home', 'GD_home', "W_home", 
                "Pts_home", 'rank_away', 'GD_away', 
                "W_away", "Pts_away"]
target = "result"

Now, we can used all the functions and variables defined before to train and predict our Random Forest model.

In [96]:
seasons = parse_seasons(seasons)
training_data = df_train(seasons)

In [97]:
training_data

,season,division,matchday,date,time,home_team,away_team,score,result,rank_home,...,T_home,Pts_home,rank_away,GF_away,GA_away,GD_away,W_away,L_away,T_away,Pts_away
0,2000-2001,1,1,9/9/00,8:15 PM,Real Sociedad,Racing,2:2,X,10.0,...,1.0,1.0,9.0,2.0,2.0,0.0,0.0,0.0,1.0,1.0
1,2000-2001,1,1,9/9/00,9:00 PM,Real Zaragoza,Espanyol,1:2,2,14.0,...,0.0,0.0,6.0,2.0,1.0,1.0,1.0,0.0,0.0,3.0
2,2000-2001,1,1,9/9/00,9:00 PM,Barcelona,Málaga CF,2:1,1,5.0,...,0.0,3.0,13.0,1.0,2.0,-1.0,0.0,1.0,0.0,0.0
3,2000-2001,1,1,9/9/00,9:00 PM,Dep. La Coruña,Athletic,2:0,1,4.0,...,0.0,3.0,17.0,0.0,2.0,-2.0,0.0,1.0,0.0,0.0
4,2000-2001,1,1,9/9/00,9:00 PM,Real Madrid,Valencia,2:1,1,7.0,...,0.0,3.0,15.0,1.0,2.0,-1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8415,2009-2010,2,42,6/19/10,6:00 PM,Cádiz CF,CD Numancia,4:2,1,20.0,...,14.0,50.0,8.0,55.0,53.0,2.0,16.0,15.0,11.0,59.0
8416,2009-2010,2,42,6/19/10,6:00 PM,Celta de Vigo,SD Huesca,0:1,2,14.0,...,13.0,52.0,13.0,36.0,40.0,-4.0,12.0,14.0,16.0,52.0
8417,2009-2010,2,42,6/19/10,6:00 PM,Elche CF,Real Sociedad,4:1,1,6.0,...,9.0,63.0,1.0,53.0,37.0,16.0,20.0,8.0,14.0,74.0
8418,2009-2010,2,42,6/19/10,6:00 PM,UD Las Palmas,Gimnàstic,1:0,1,17.0,...,15.0,51.0,18.0,42.0,55.0,-13.0,14.0,19.0,9.0,51.0


In [98]:
X_train = training_data[features]  
y_train = training_data[target]  

We have used the function `GridSearchCV` to find the best estimator to the Random Forest Classifier in each execution. Because of that, the training is going to take more time than if we have set the parameters directly but the accuracy is going to be higher. However, it does not take too much time. To train 10 seasons it takes around one minute and a half, and to train all the seasons it takes around 6 minutes. 

In [99]:
rf_classifier = RandomForestClassifier()
    
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

rf_model = grid_search.best_estimator_

rf_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, min_samples_leaf=2, min_samples_split=5,
                       n_estimators=50)

In [ ]:
test_data = df_test(season, division, matchday)
test_data

In [100]:
X_test = test_data[features]
y_test = test_data[target]

In [101]:
y_pred = rf_model.predict(X_test)
y_pred

array(['1', '1', 'X', '1', 'X', '2', '1', 'X', '1', '1'], dtype=object)

Now, we can mesure the fitnees of our model. In this example the accuracy is high, 0.7.

The diagonal of the confusion matrix indicates how many predictions has done correctly of each class. For example, in this case, it has predicted corectly 5 results of class (1) but it has get wrong one. It has predicted a tie (X) when it was (1).



In [102]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
predictions = rf_model.predict_proba(X_test)

print(f"Accuracy: {accuracy}\n")
print(f"Confusion Matrix:\n{confusion_mat}\n")
print(f"Classification Report:\n{class_report}\n")
print(f"Predictions probability:\n{predictions}\n")

Accuracy: 0.7

Confusion Matrix:
[[5 0 0]
 [0 1 2]
 [1 0 1]]

Classification Report:
              precision    recall  f1-score   support

           1       0.83      1.00      0.91         5
           2       1.00      0.33      0.50         3
           X       0.33      0.50      0.40         2

    accuracy                           0.70        10
   macro avg       0.72      0.61      0.60        10
weighted avg       0.78      0.70      0.68        10


Predictions probability:
[[8.23033482e-01 2.80392200e-02 1.48927298e-01]
 [9.67924616e-01 6.59340659e-04 3.14160433e-02]
 [5.47166594e-03 4.25025494e-01 5.69502840e-01]
 [7.43366023e-01 5.17896049e-02 2.04844372e-01]
 [4.00856611e-02 4.52731183e-01 5.07183156e-01]
 [1.94062262e-02 5.43513524e-01 4.37080250e-01]
 [9.69539354e-01 2.28063800e-03 2.81800075e-02]
 [1.28215266e-02 2.26459997e-01 7.60718476e-01]
 [8.12983260e-01 3.20969845e-02 1.54919755e-01]
 [5.52327346e-01 4.81954732e-02 3.99477181e-01]]



# Total Accuracy

In [115]:
seasons2 = "all"

In [116]:
seasons2 = parse_seasons(seasons2)
training_data2 = df_train(seasons2)

In [117]:
X_train2 = training_data2[features]
y_train2 = training_data2[target]

In [119]:
rf_classifier = RandomForestClassifier()
    
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train2, y_train2)

rf_model2 = grid_search.best_estimator_

rf_model2.fit(X_train2, y_train2)

KeyboardInterrupt: 

In [112]:
test_data2 = df_train(seasons2)
X_test2 = test_data2[features]
y_test2 = test_data2[target]

In [113]:
y_pred2 = rf_model2.predict(X_test2)
y_pred2

array(['2', '1', '1', ..., '2', 'X', '2'], dtype=object)

In [114]:
# Evaluate the model
accuracy2 = accuracy_score(y_test2, y_pred2)
confusion_mat2 = confusion_matrix(y_test2, y_pred2)
class_report2 = classification_report(y_test2, y_pred2)
predictions2 = rf_model2.predict_proba(X_test2)

print(f"Accuracy: {accuracy2}\n")
print(f"Confusion Matrix:\n{confusion_mat2}\n")
print(f"Classification Report:\n{class_report2}\n")
print(f"Predictions probability:\n{predictions2}\n")

Accuracy: 0.6388125

Confusion Matrix:
[[23671  1408   238]
 [ 5206  4855   342]
 [ 8236  1907  2137]]

Classification Report:
              precision    recall  f1-score   support

           1       0.64      0.93      0.76     25317
           2       0.59      0.47      0.52     10403
           X       0.79      0.17      0.28     12280

    accuracy                           0.64     48000
   macro avg       0.67      0.53      0.52     48000
weighted avg       0.67      0.64      0.59     48000


Predictions probability:
[[4.27753141e-03 9.25900410e-01 6.98220589e-02]
 [9.97216173e-01 1.42663962e-04 2.64116270e-03]
 [9.98342445e-01 1.80306275e-04 1.47724873e-03]
 ...
 [1.43315574e-02 5.03302859e-01 4.82365584e-01]
 [2.31576547e-01 1.19444130e-01 6.48979323e-01]
 [0.00000000e+00 8.29649466e-01 1.70350534e-01]]

